## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook
for the following questions.


In [15]:
# Establish connection with SQLite Database
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///data/country_club_db.sqlite')

In [22]:
# snapshot of Members table
df_members = pd.read_sql_query("SELECT * FROM Members", engine)
df_members.head(2)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05


In [24]:
# snapshot of Facilities table
df_facilities = pd.read_sql_query("SELECT * FROM Facilities", engine)
df_facilities.head(2)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200


In [25]:
# snapshot of Bookings table
df_Bookings = pd.read_sql_query("SELECT * FROM Bookings", engine)
df_Bookings.head(2)

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2



### QUESTIONS:
**Q10**: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!


In [75]:
# Construct Query
query_10 = '''
SELECT name 
       , tot_revenue 
FROM   (SELECT f.name 
               , Sum(CASE 
                       WHEN b.memid = 0 THEN f.guestcost * b.slots 
                       ELSE f.membercost * b.slots 
                     end) AS tot_revenue 
        FROM   bookings AS b 
               INNER JOIN facilities AS f 
                       ON b.facid = f.facid 
        GROUP  BY f.name) 
WHERE  tot_revenue > 1000 
ORDER  BY tot_revenue DESC; 
'''

In [76]:
# Execute Query
q10 = pd.read_sql_query(query_10, engine)

In [77]:
# Display Result
q10

,name,tot_revenue
0,Massage Room 1,50351.6
1,Massage Room 2,14454.6
2,Tennis Court 2,14310.0
3,Tennis Court 1,13860.0
4,Squash Court,13468.0
5,Badminton Court,1906.5


**Q11**: Produce a report of members and who recommended them in alphabetic surname,firstname order


In [78]:
# Construct Query
query_11 = '''
SELECT name 
       , tot_revenue 
FROM   (SELECT f.name 
               , Sum(CASE 
                       WHEN b.memid = 0 THEN f.guestcost * b.slots 
                       ELSE f.membercost * b.slots 
                     end) AS tot_revenue 
        FROM   bookings AS b 
               INNER JOIN facilities AS f 
                       ON b.facid = f.facid 
        GROUP  BY f.name) 
WHERE  tot_revenue > 1000 
ORDER  BY tot_revenue DESC; 
'''

In [79]:
# Execute Query
q11 = pd.read_sql_query(query_10, engine)

In [ ]:
# Display Result
q10